In [9]:
import pandas as pd
import numpy as np
import time
import datetime
import os

N_STATES = 5   # ['action','close-avg','deal','amp_cover','vol_cover'] 
ACTIONS_Strategy = 
ACTIONS = ['Wait','Buy','Sell','OpenInteres','close-avg','deal','amp_cover','vol_cover'] #行為 空手、買進、賣出、持有、多單平倉、空單平倉
# ACTIONS = ['action','close-avg','deal','amp_cover','vol_cover']     # 探索者的可用动作
EPSILON = 0.9   # 贪婪度 greedy
ALPHA = 0.1     # 学习率
GAMMA = 0.9    # 奖励递减值
MAX_EPISODES = 13   # 最大回合数
FRESH_TIME = 0.3    # 移动间隔时间
global OpenInteres
OpenInteres = 0 # 是否持倉中
global priceAtPurchase #損益
priceAtPurchase = 0

In [12]:
def get_data():
    direct=os.path.abspath('../data')
    filelist = os.listdir('../data')
    file = filelist[-1]
    print(file)
    dayticks = pd.read_csv(direct+'\\'+file,header=None,names=['ndatetime','nbid','nask','close','volume','deal'])
    dayticks['ndatetime'] = pd.to_datetime(dayticks['ndatetime'], format='%Y-%m-%d %H:%M:%S.%f')
    dayticks.sort_values(by=['ndatetime'],ascending=True)
    dayticks=dayticks[(dayticks.ndatetime.dt.hour>=8) & (dayticks.ndatetime.dt.hour<15)]
    dayticks.index = dayticks.ndatetime
    Candledf=dayticks['close'].resample('1min',closed='right').ohlc()
    tmpdf=dayticks['volume'].resample('1min').sum()
    Candledf=pd.concat([Candledf,tmpdf],axis=1)
    del tmpdf
    tmpdf=dayticks['deal'].resample('1min').sum()
    Candledf=pd.concat([Candledf,tmpdf],axis=1)
    del tmpdf
    Candledf['dealcumsum']=Candledf['deal'].cumsum()
    del Candledf['deal']
    Candledf.rename(columns={'dealcumsum':'deal'},inplace=True)
    Candledf=Candledf.rename_axis('ndatetime').reset_index()
    Candledf['ndatetime'] = pd.to_datetime(Candledf['ndatetime'], format='%Y-%m-%d %H:%M:%S.%f')
    Candledf.sort_values(by=['ndatetime'],ascending=True)
    tmpline=Candledf.close.cumsum()
    Candledf['Avg_line'] = tmpline.apply(lambda x: x/(tmpline[tmpline==x].index[0]+1))
    del tmpline
    Candledf['amplitude'] = Candledf.high - Candledf.low
    Candledf['close_Avg'] = Candledf.close - Candledf.Avg_line
    Candledf['close_diff'] = Candledf.close-Candledf.close.shift(1).fillna(Candledf.close)
    Candledf['close_diff'] = Candledf.close_diff.shift(-1).fillna(0)
    global CandleData
    CandleData = Candledf[['ndatetime','close_Avg','deal','amplitude','volume','close','close_diff']]
    print(CandleData.tail())
    return CandleData

In [28]:
# data = get_data()
QT = pd.DataFrame(np.zeros((1,len(ACTIONS_Strategy))),columns = ACTIONS_Strategy,dtype=np.float32,name = 'Wait')
print(QT)
# QT=QT.append(pd.Series([0]*len(ACTIONS),index=ACTIONS,name='state'))
# print(QT.loc['state',QT.columns])

TypeError: __init__() got an unexpected keyword argument 'name'

In [31]:
class QLearning:
    def __init__(self,action_strategy=None,actionslist=None,LearnRate=0.01,disscount_factory=0.9,e_greedy=0.1):
        self.actions = actionslist # action 列表
        self.strategy =['Action','OpenInteres','close-avg','deal','amp_cover','vol_cover']
        self.strategy_action = ['Wait','Buy','Sell','Short','BuyCover','ShortStopLoss','BuyCoverStopLoss'] #行為 空手、買進、賣出、持有、多單平倉、空單平倉
        self.lr = LearnRate #學習效率
        self.gamma = disscount_factory # 折扣因子
        self.espilon = e_greedy #貪婪度
        self.q_strategy = pd.DataFrame(columns=self.strategy,dtype=np.float32) #建立空Q Table
        self.q_table = pd.DataFrame(columns=self.actions,dtype=np.float32) #建立空Q Table
        self.openInteres = 0
        
    def check_state_exist(self,state):
        if state.name not in self.q_table.index:
            self.q_table = self.q_table.append(pd.Series([0]*len(self.actions[:4])+state.to_list()
            ,index=self.q_table.columns,name=state.name))
    
    def choose_action(self,state):
        self.check_state_exist(state)
        #隨機選行為 Epsilon Greedy貪婪方法
        if np.random.uniform()>self.espilon:
        # if np.random.uniform()>1:
            #隨機選 1個action
            action = np.random.choice(self.actions[:3])
        else: #取Q值最高的action
            # print(state.name)
            # self.q_table.loc[state.name,'Wait'] = 1
            state_action = self.q_table.loc[state.name,['Wait','Buy','Sell']]
            # state_action = state_action.reindex(np.random.permutation(state_action.index))
            # 取當下最大Q值
            # print(state_action[state_action == state_action.max()].index,'動作列')
            action = state_action.index[state_action == state_action.max()].to_list()            
            action = str(action[0])
            # action = state_action.idxmax()
        return action
    # Q Table 學習
    def learn(self,s,a,r,s_): #state,action,result,s_(下個state)
        #檢查下個 state
        self.check_state_exist(s_) #檢查s_ 是否在q_table
        # Q(S,A) <- Q(S,A)+a*[R+v*max(Q(S',a)-Q(S,A))]
        q_predict = self.q_table.loc[s.name,a] #根據Q表 得到 預估值 predict
        #q_target 現實值
        if s_.name != -100: #下一步 不是最終
            q_target = r + self.gamma * self.q_table.loc[s_.name,['Wait','Buy','Sell']].max()
            
        else: # 下一步終結
            q_target = r 
        # 更新Q表
        self.q_table.loc[s.name,a] += self.lr * (q_target - q_predict)

In [32]:
# https://blog.csdn.net/joinquantdata/article/details/87254611?spm=1001.2101.3001.6650.6&utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7Edefault-6.no_search_link&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7Edefault-6.no_search_link
class Market:
    def __init__(self,data) -> None:
        self.action_list = ['Wait','Buy','Sell','OpenInteres','ndatetime','close_Avg','deal','amplitude','volume','close','close_diff'] # 買進、賣出、觀望
        self.actions = len(self.action_list)
        self.data = data # 1分鐘 K線資料
        self.idx = 10 #預設開始跑策略的K線
        self.time = self.data.at[self.idx,'ndatetime'].time().strftime('%H:%M')
        self.OpenInterest = 0

    def get_state(self,idx):
        
        return self.data.loc[idx,:] #ndatetime  close_Avg  deal  amplitude  volume  close  close_diff

    def step(self,idx,action):
        state = self.get_state(idx)
        ntime = state['ndatetime'].time().strftime('%H:%M:%S')
        if ntime == '13:20:00':
            done = True
            reward = 0
            s_ = -100
        else:
            done = False
            reward = self.data.at[idx,'close_diff']
            s_ = idx + 1

        if action == 'Buy':
            pass
        if action == 'Sell':
            reward = reward * -1
        if action == 'Wait':
            reward = 0
        if s_ != -100:
            s_ = idx + 1
    
        return s_,reward,done

    def reset(self):
        self.idx=10
        return self.idx
def update(CandleData):
    evn = Market(CandleData)
    RL = QLearning(evn.action_list)
    
    for episode in range(50):
        idx = evn.reset()
        step_count = 0
        while True:
            state = evn.get_state(idx)
            action = RL.choose_action(state)
            idx_,reward,done = evn.step(idx,action)
            if done:
                break
            else: 
                state_ = evn.get_state(idx_)
                # step_count+=1
                RL.learn(state,action,reward,state_)
                idx = idx_
            # print(idx,action)
    return RL.q_table

In [35]:
data = get_data()
# q_table = update(data)
# print(q_table.tail())

evn = Market(data)
state = evn.get_state(10)
print(state)
RL = QLearning(ACTIONS_Strategy,evn.action_list)
RL.check_state_exist(state)
action = RL.choose_action(state)
print(action)
# print(RL.q_table.loc[10,['Wait','Buy','Sell']])

Ticks2022-01-04.txt
              ndatetime  close_Avg  deal  amplitude  volume  close  close_diff
295 2022-01-04 13:40:00  52.003378  6078          4     480  18512         1.0
296 2022-01-04 13:41:00  52.824916  6081          3     311  18513        -3.0
297 2022-01-04 13:42:00  49.657718  5936          5     511  18510        -1.0
298 2022-01-04 13:43:00  48.494983  5904          3     410  18509         7.0
299 2022-01-04 13:44:00  55.310000  6154          9    1062  18516         0.0
ndatetime     2022-01-04 08:55:00
close_Avg                4.818182
deal                          507
amplitude                       3
volume                        111
close                       18379
close_diff                    9.0
Name: 10, dtype: object
Wait


In [ ]:
print(q_table.Buy.sum(),q_table.Sell.sum(),q_table.close_diff.sum())